In [1]:
import psycopg2
import requests
import pandas as pd
import json
from sqlalchemy import create_engine

In [95]:
conn = psycopg2.connect(
    database='riot',
    user='admin',
    password='admin',
    host='db',
    port='5432'
)

engine = create_engine('postgresql+psycopg2://', creator=lambda: conn)

In [90]:
cursor = conn.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)

('test',)
('dim_champions',)
('matchid_queue',)


In [4]:
champions_url = 'https://ddragon.leagueoflegends.com/cdn/13.22.1/data/en_US/champion.json'
res = requests.get(champions_url)
res_dict = json.loads(res.text)
champion_df_raw = pd.DataFrame(res_dict['data']).T
champion_df = champion_df_raw.set_index('key', drop=True)

In [5]:
champion_df.head()

,version,id,name,title,blurb,info,image,tags,partype,stats
key,,,,,,,,,,
266,13.22.1,Aatrox,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'full': 'Aatrox.png', 'sprite': 'champion0.pn...","[Fighter, Tank]",Blood Well,"{'hp': 650, 'hpperlevel': 114, 'mp': 0, 'mpper..."
103,13.22.1,Ahri,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,"{'attack': 3, 'defense': 4, 'magic': 8, 'diffi...","{'full': 'Ahri.png', 'sprite': 'champion0.png'...","[Mage, Assassin]",Mana,"{'hp': 590, 'hpperlevel': 96, 'mp': 418, 'mppe..."
84,13.22.1,Akali,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,"{'attack': 5, 'defense': 3, 'magic': 8, 'diffi...","{'full': 'Akali.png', 'sprite': 'champion0.png...",[Assassin],Energy,"{'hp': 570, 'hpperlevel': 119, 'mp': 200, 'mpp..."
166,13.22.1,Akshan,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...","{'attack': 0, 'defense': 0, 'magic': 0, 'diffi...","{'full': 'Akshan.png', 'sprite': 'champion0.pn...","[Marksman, Assassin]",Mana,"{'hp': 630, 'hpperlevel': 107, 'mp': 350, 'mpp..."
12,13.22.1,Alistar,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"{'attack': 6, 'defense': 9, 'magic': 5, 'diffi...","{'full': 'Alistar.png', 'sprite': 'champion0.p...","[Tank, Support]",Mana,"{'hp': 685, 'hpperlevel': 120, 'mp': 350, 'mpp..."


In [55]:
info_unpacked = champion_df.apply(lambda x: pd.Series(x['info']), axis=1, result_type='expand')
stats_unpacked = champion_df.apply(lambda x: pd.Series(x['stats']), axis=1, result_type='expand')
champion_df = champion_df \
                .join(info_unpacked) \
                .join(stats_unpacked)

champion_df.head()

,version,id,name,title,blurb,info,image,tags,partype,stats,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
key,,,,,,,,,,,,,,,,,,,,,
266,13.22.1,Aatrox,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'full': 'Aatrox.png', 'sprite': 'champion0.pn...","[Fighter, Tank]",Blood Well,"{'hp': 650, 'hpperlevel': 114, 'mp': 0, 'mpper...",...,3.00,1.00,0.0,0.0,0.0,0.0,60.0,5.00,2.500,0.651
103,13.22.1,Ahri,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,"{'attack': 3, 'defense': 4, 'magic': 8, 'diffi...","{'full': 'Ahri.png', 'sprite': 'champion0.png'...","[Mage, Assassin]",Mana,"{'hp': 590, 'hpperlevel': 96, 'mp': 418, 'mppe...",...,2.50,0.60,8.0,0.8,0.0,0.0,53.0,3.00,2.200,0.668
84,13.22.1,Akali,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,"{'attack': 5, 'defense': 3, 'magic': 8, 'diffi...","{'full': 'Akali.png', 'sprite': 'champion0.png...",[Assassin],Energy,"{'hp': 570, 'hpperlevel': 119, 'mp': 200, 'mpp...",...,9.00,0.90,50.0,0.0,0.0,0.0,62.0,3.30,3.200,0.625
166,13.22.1,Akshan,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...","{'attack': 0, 'defense': 0, 'magic': 0, 'diffi...","{'full': 'Akshan.png', 'sprite': 'champion0.pn...","[Marksman, Assassin]",Mana,"{'hp': 630, 'hpperlevel': 107, 'mp': 350, 'mpp...",...,3.75,0.65,8.2,0.7,0.0,0.0,52.0,3.00,4.000,0.638
12,13.22.1,Alistar,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"{'attack': 6, 'defense': 9, 'magic': 5, 'diffi...","{'full': 'Alistar.png', 'sprite': 'champion0.p...","[Tank, Support]",Mana,"{'hp': 685, 'hpperlevel': 120, 'mp': 350, 'mpp...",...,8.50,0.85,8.5,0.8,0.0,0.0,62.0,3.75,2.125,0.625


In [5]:
root_names = ['4l3c4', 'Riot%20GalaxySmash', 'weirdosuper2']
puuids = []

for n in root_names:
    url = f'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{n}'
    res = requests.get(
        url=url,
        headers={'X-Riot-Token':'RGAPI-201745c1-4613-484c-bcab-8d3b689bb734'}
    )
    res_json = json.loads(res.text)
    puuids.append(res_json['puuid'])



In [17]:
puuids

['PA_fBneBfDsMObRXAMShLREa1oNspSFIjrlYOPjm5TwaYYMuaso5TUb51NtlkyuZWhft7UQFzo1kmQ',
 'ht02PM3UgviRbznWJf2xa0xQc_Ls7O7kyvXTbgn98rX0NMZsd3TsqyTh_Q1sHJs1U8eZzrg4wxNlvA',
 'KIbW5DcJRvAa5pbKwctIF9HGURlwKJ-FBgOZNIldrY6QhO_5eNpDdgGP9AaEF0XML0vZAEszJHzddA']

In [22]:
url = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/PA_fBneBfDsMObRXAMShLREa1oNspSFIjrlYOPjm5TwaYYMuaso5TUb51NtlkyuZWhft7UQFzo1kmQ/ids'
res = requests.get(
    url=url,
    params={
        'startTime': '1696888052',
        'endTime': '1699570052',
        'type': 'ranked',
        'start': '0',
        'count': '100'
    },
    headers={'X-Riot-Token':'RGAPI-201745c1-4613-484c-bcab-8d3b689bb734'}
)
match_ids = res.json()

In [116]:
# api functions
def get_match_ids(puuid):
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids'
    res = requests.get(
        url=url,
        params={
            'startTime': '1696888052',
            'endTime': '1699570052',
            'type': 'ranked',
            'start': '0',
            'count': '100'
        },
        headers={'X-Riot-Token':'RGAPI-201745c1-4613-484c-bcab-8d3b689bb734'}
    )
    match_ids = res.json()

    return match_ids

def get_match_data(match_id):
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}'
    res = requests.get(
        url=url,
        headers={'X-Riot-Token':'RGAPI-201745c1-4613-484c-bcab-8d3b689bb734'}
    )
    match_df = pd.DataFrame(res.json()['info']['participants'])
    match_df['matchID'] = match_id
    match_df = match_df[[
        'matchID',
        'puuid',
        'championId',
        'teamPosition',
        'win'
    ]].copy()

    return match_df

def get_champion_mastery(puuid, championID):
    url = f'https://na1.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}/by-champion/{championID}'
    res = requests.get(
        url=url,
        headers={'X-Riot-Token':'RGAPI-201745c1-4613-484c-bcab-8d3b689bb734'}
    )
    return res.json()

# db functions
def upload_match_ids_to_queue(match_ids, conn):
    cursor = conn.cursor()
    for m in match_ids:
        sql = f"""INSERT INTO matchID_queue (matchid) VALUES ('{m}') ON CONFLICT DO NOTHING;"""
        cursor.execute(sql)
    conn.commit()
    cursor.close()

    return

def delete_match_id_from_queue(match_id, conn):
    cursor = conn.cursor()
    for m in match_ids:
        sql = f"""DELETE FROM matchID_queue WHERE matchid = '{match_id}';"""
        cursor.execute(sql)
    conn.commit()
    cursor.close()

    return

def upload_champion_mastery(mastery_dict, conn):
    puuid = mastery_dict['puuid']
    championid = mastery_dict['championId']
    points = mastery_dict['championPoints']

    cursor = conn.cursor()
    sql = f"""
        INSERT INTO dim_champion_mastery(puuid, championId, championPoints) 
        VALUES ('{puuid}', {championid}, {points})
        ON CONFLICT DO NOTHING
        ;
    """
    cursor.execute(sql)
    conn.commit()
    cursor.close()

In [107]:
get_match_data('NA1_4809203541')['puuid'].values

array(['6q5hj4OzjCAznK2aqU9NRFO6S_vUU9z_krleRbbHyDyI019hKoaXlV20IPLLGKwYwlu0OHYoZojFNg',
       'PA_fBneBfDsMObRXAMShLREa1oNspSFIjrlYOPjm5TwaYYMuaso5TUb51NtlkyuZWhft7UQFzo1kmQ',
       'SmAuRxCOZ0NQlfcbtY-hvE8qygKuvXdBxWdUzmmpNtstxSggrsKqBelIIxBek9AqmIWDZyU4eUCJ8A',
       '9qWrklGlrY_j__ztgwfhK4Q9V8ri3f1xHDb6ANN41rP91yel3o0WA1kQyBcwhGwAUzzGw-3QIY78ng',
       'fI7ABihwPi56xWChEff6uO0Vi02PcYea6JLEMdAcIUp4QcesOIKzkqz9g6fXmNjWliktnT0upQOwJw',
       'EFc_RXpXDvlFLwwrSv9mokm_vdhkXaIc9v7eaBrcnf1ZMaAxAyTl_vVNYCRvhXxt8ilHjRhzG-v9Ag',
       'jnu8vguGSf5Uz3LvMeefgAIQnaFk2N275T8WIO-ZrHqaG3qIgfPzxsjzbd5JGtCACm7X9R2NzCkjpQ',
       'kyY9GEjXqEsk0F_YkVrph_1mHIn4DtxpoANPvGhR2hqdZT3lzMyrhTbot_ZGRL-SuhoP0hgNJKCZ8g',
       'NUEMkEZxDK5Wz5ltfNSQJ4m6J4t6T5sMyaeTju68OHzNmArKYezhkyyQIH2zGjDwmrnoMW8jtlr-1A',
       'BGaVnaJdouKmTbHZolbeRc1rfWULQXtZADJms0OvrznQJx8A4_2e_T0gAF7f5xQlCnI6fTDfwxsgjA'],
      dtype=object)

In [113]:
get_champion_mastery('6q5hj4OzjCAznK2aqU9NRFO6S_vUU9z_krleRbbHyDyI019hKoaXlV20IPLLGKwYwlu0OHYoZojFNg', 24)

,puuid,championId,championPoints
0,6q5hj4OzjCAznK2aqU9NRFO6S_vUU9z_krleRbbHyDyI01...,24,178186


In [117]:
match_ids

['NA1_4812312011',
 'NA1_4810221562',
 'NA1_4810169301',
 'NA1_4809203541',
 'NA1_4809145513',
 'NA1_4809120245',
 'NA1_4809105941',
 'NA1_4809077046',
 'NA1_4804582547',
 'NA1_4797633371',
 'NA1_4797604818']

In [118]:
for m in match_ids:
    print(m)
    
    # add match id to queue

    # get match info (1 request)
    match_df = get_match_data(m)
    
    # upload required match data
    match_df.to_sql(
        name='fct_matches',
        con=engine,
        if_exists='append',
        index=False,
        method='multi'
    )

    # get mastery info (10 requests)
    for puuid, cid in match_df[['puuid', 'championId']].values:
        mastery_dict = get_champion_mastery(puuid, cid)
    
        # upload to mastery table
        upload_champion_mastery(mastery_dict, conn)

    # get matches from players (10 requests)
    for puuid in match_df['puuid'].values:
        # get matches from puuid
        match_ids = get_match_ids(puuid)

        # add to match id queue
        upload_match_ids_to_queue(match_ids, conn)

    # remove match id from queue
    delete_match_id_from_queue(m, conn)

    break


NA1_4812312011


In [60]:
fopud, chid = df[['puuid', 'championId']].values

In [61]:
pud

'n6eyh6rK44Cf7OKeN9mr06_9Ejjsob--6VlKwRahNaKMB4VIZ8t6gUjRbMwj1dsdDL3eY02bLMcMcg'

In [52]:
df = pd.DataFrame(test['info']['participants'])
df['matchID'] = m
df[[
    'matchID',
    'puuid',
    'championId',
    'teamPosition',
    'win'
]]

,matchID,puuid,championId,teamPosition,win
0,NA1_4812312011,n6eyh6rK44Cf7OKeN9mr06_9Ejjsob--6VlKwRahNaKMB4...,420,TOP,True
1,NA1_4812312011,PA_fBneBfDsMObRXAMShLREa1oNspSFIjrlYOPjm5TwaYY...,245,JUNGLE,True
2,NA1_4812312011,gPuj9yaZoMwNVaU4f8Vouu5WoNDjddixh-O-hDJq4YUDS9...,3,MIDDLE,True
3,NA1_4812312011,j9Yf1M13BHw1BO2EZzk4DOp1I7PJtZAni7RC238QMJGqIt...,18,BOTTOM,True
4,NA1_4812312011,KWrk6z-QuDeH4ML0dwg0b7ruaKxVcLR3n78qiw83Qxz3uJ...,117,UTILITY,True
5,NA1_4812312011,Co7vLlpNY79FzfHSNjjdgAgOiBJb7QXwDayFgpU33q-she...,68,TOP,False
6,NA1_4812312011,1m9-34YifBdFpE6RgY3_zxTvm2po3KdkPWhuEHgk388gww...,9,JUNGLE,False
7,NA1_4812312011,Q1FhxznX5FbL7J6KXl4LB4_sCw2F1_5S9APm8eDL6S0j94...,166,MIDDLE,False
8,NA1_4812312011,cV22n1Jd29VZnOBd76LoGjGdH1YDmMme26IBlSssa68tOv...,21,BOTTOM,False
9,NA1_4812312011,bsc5zyQfxQYJvr4-Npw_C7-BsJhQ4jdl7gaFoduYXbtTKZ...,412,UTILITY,False


In [51]:
test['info']['participants'][0].keys()

dict_keys(['allInPings', 'assistMePings', 'assists', 'baitPings', 'baronKills', 'basicPings', 'bountyLevel', 'challenges', 'champExperience', 'champLevel', 'championId', 'championName', 'championTransform', 'commandPings', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets', 'damageSelfMitigated', 'dangerPings', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'eligibleForProgression', 'enemyMissingPings', 'enemyVisionPings', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'getBackPings', 'goldEarned', 'goldSpent', 'holdPings', 'individualPosition', 'inhibitorKills', 'inhibitorTakedowns', 'inhibitorsLost', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'itemsPurchased', 'killingSprees', 'kills', 'lane', 'largestCriticalStrike', 'largestKillingSpree', 'largestMultiKill', 'longestTimeSpentLiving', 'magicDamageDealt', 'magicDamageDe

In [76]:
test = [1,2,3,4,5]

for i in test:
    test = [1,2,3,4,5,i+5]
    test.remove(i)
    print(i)

1
2
3
4
5


In [77]:
test

[1, 2, 3, 4, 10]